# ConversationKGMemory

Không giống như `ConversationEntityMemory`, vốn quản lý thông tin về các entities theo định dạng key-value cho từng entity riêng lẻ, `ConversationKGMemory` (Conversation Knowledge Graph Memory) là một module quản lý các relationships giữa các entities theo định dạng graph.

Nó trích xuất và cấu trúc các **knowledge triplets** (subject-relationship-object) để xác định và lưu trữ các relationships phức tạp giữa các entities, và cho phép khám phá entity connectivity thông qua **graph structure**.

Điều này giúp model hiểu được các relationships giữa các entities khác nhau và phản hồi tốt hơn cho các queries dựa trên complex networks và historical context.


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Conversation Knowlege Graph Memory

`ConversationKGMemory` là một memory module lưu trữ và quản lý thông tin được trích xuất từ các conversations trong một graph structure. Ví dụ này minh họa các tính năng chính sau:

- Lưu trữ conversation context (`save_context`)
- (Tham khảo) Lấy danh sách tên các entities trong graph được sắp xếp theo causal dependence. (`get_topological_sort`)
- Trích xuất các entities từ current conversation (`get_current_entities`)
- Trích xuất knowledge triplets (`get_knowledge_triplets`)
- Truy xuất memory đã lưu trữ (`load_memory_variables`)

Ví dụ sau đây cho thấy quá trình trích xuất các entities và relationships từ một conversation về một nhà thiết kế mới, Shelly Kim, và lưu trữ chúng ở định dạng graph.


In [2]:
from langchain_community.memory.kg import ConversationKGMemory

memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "This is Shelly Kim who lives in Pangyo."},
    {"output": "Hello Shelly, nice to meet you! What kind of work do you do?"},
)
memory.save_context(
    {"input": "Shelly Kim is our company's new designer."},
    {
        "output": "That's great! Welcome to our team. I hope you'll enjoy working with us."
    },
)

### (Reference) get_knowledge_triplets(input_string: str) → List[KnowledgeTriple]

Bạn có thể sử dụng method `get_topological_sort` để xem tất cả các entities được lưu trữ trong knowledge graph theo topological order:

Method này:
- Sử dụng thư viện NetworkX để phân tích knowledge graph structure
- Thực hiện topological sorting dựa trên các directed edges
- Trả về danh sách các entities theo dependency order

Thứ tự phản ánh các relationships giữa các entities trong conversation, cho thấy chúng được kết nối như thế nào trong knowledge graph.


In [3]:
memory.kg.get_topological_sort()

['Shelly Kim', 'Pangyo', "company's new designer"]

### `get_current_entities(input_string: str) → List[str]`

Đây là cách method `get_current_entities` hoạt động:

**1. Tạo Entity Extraction Chain**
- Tạo một `LLMChain` sử dụng template `entity_extraction_prompt`.
- Prompt này được thiết kế để trích xuất các proper nouns từ dòng cuối cùng của conversation.

**2. Xử lý Context**
- Truy xuất **k*2** messages cuối cùng từ buffer. (mặc định: k=2)
- Tạo chuỗi conversation history sử dụng `human_prefix` và `ai_prefix`.

**3. Trích xuất Entity**
- Trích xuất các proper nouns từ chuỗi đầu vào "Who is Shelly Kim?".
- Chủ yếu nhận dạng các từ bắt đầu bằng chữ cái viết hoa là proper nouns.
- Trong trường hợp này, "Shelly Kim" được trích xuất như một entity.

Method này **chỉ trích xuất các entities từ chính câu hỏi**, trong khi conversation context trước đó chỉ được sử dụng để tham khảo.


In [4]:
memory.get_current_entities({"input": "Who is Shelly Kim?"})

['Shelly Kim', 'Pangyo']

### `get_knowledge_triplets(input_string: str) → List[KnowledgeTriple]`

Method `get_knowledge_triplets` hoạt động như sau:

**1. Knowledge Triple Extraction Chain**
- Tạo một `LLMChain` sử dụng template `knowledge_triplet_extraction_prompt`.
- Được thiết kế để trích xuất các triples theo định dạng (**subject-relation-object**) từ văn bản đã cho.

**2. Tìm kiếm Memory**
- Tìm kiếm thông tin liên quan đến "Shelly" từ các conversations đã lưu trữ trước đó.
- Context đã lưu trữ:
  - "This is Shelly Kim who lives in Pangyo."
  - "Shelly Kim is our company's new designer."

**3. Trích xuất Triple**
- Tạo các triples sau từ thông tin đã truy xuất:
  - (Shelly Kim, lives in, Pangyo)
  - (Shelly Kim, is, designer)
  - (Shelly Kim, works at, our company)

Method này trích xuất thông tin relationship ở **định dạng triple** từ tất cả nội dung conversation đã lưu trữ **liên quan đến một entity cụ thể**.


In [5]:
memory.get_knowledge_triplets({"input": "Shelly"}), "\n", memory.get_knowledge_triplets(
    {"input": "Pangyo"}
), "\n", memory.get_knowledge_triplets(
    {"input": "designer"}
), "\n", memory.get_knowledge_triplets(
    {"input": "Langchain"}
)

([KnowledgeTriple(subject='Shelly Kim', predicate='lives in', object_='Pangyo')],
 '\n',
 [KnowledgeTriple(subject='Shelly Kim', predicate='lives in', object_='Pangyo')],
 '\n',
 [KnowledgeTriple(subject='Shelly Kim', predicate='is', object_='designer')],
 '\n',
 [])

In [6]:
memory.get_knowledge_triplets({"input": "Shelly"})

[KnowledgeTriple(subject='Shelly Kim', predicate='lives in', object_='Pangyo')]

In [7]:
memory.get_knowledge_triplets(
    {"input": "Pangyo"}
)

[KnowledgeTriple(subject='Shelly Kim', predicate='lives in', object_='Pangyo')]

### `load_memory_variables(inputs: Dict[str, Any]) → Dict[str, Any]`

Method `load_memory_variables` hoạt động qua các bước sau:

**1. Trích xuất Entity**
- Trích xuất các entities (ví dụ: "Shelly Kim") từ đầu vào "Who is Shelly Kim?".
- Sử dụng nội bộ method `get_current_entities`.

**2. Truy xuất Kiến thức**
- Tìm kiếm tất cả knowledge triplets liên quan đến các entities đã trích xuất.
- Truy vấn graph để tìm thông tin đã được lưu trữ trước đó thông qua `save_context`.

**3. Định dạng Thông tin**
- Chuyển đổi các triplets tìm thấy thành system messages.
- Trả về danh sách các message objects do cài đặt `return_messages=True`.

Method này truy xuất thông tin liên quan từ knowledge graph đã lưu trữ và trả về nó ở định dạng có cấu trúc, sau đó có thể được sử dụng làm context cho các conversations tiếp theo với language model.


In [8]:
memory.load_memory_variables({"input": "Who is Shelly Kim?"})

{'history': [SystemMessage(content="On Shelly Kim: Shelly Kim lives in Pangyo. Shelly Kim is company's new designer.", additional_kwargs={}, response_metadata={})]}

## Applying KG Memory to Chain

This section demonstrates how to use `ConversationKGMemory` with `ConversationChain`

(The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. If you want, you can skip to [Applying KG Memory with LCEL](#applying-kg-memory-with-lcel))

In [9]:
from langchain_community.memory.kg import ConversationKGMemory
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

/tmp/ipykernel_762349/273355205.py:19: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_with_kg = ConversationChain(


In [10]:
conversation_with_kg.predict(
    input="My name is Teddy. Shelly is a coworker of mine, and she's a new designer at our company."
)

"Hello Teddy! It's nice to meet you. Shelly sounds like a talented designer. What kind of projects is she working on at your company?"

In [11]:
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': 'On Shelly: Shelly is a coworker. Shelly is a new designer. Shelly is at our company.'}

In [12]:
conversation_with_kg.memory.clear()
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': ''}

## Applying KG Memory with LCEL

Let's examine the memory after having a conversation using a custom `ConversationChain` with `ConversationKGMemory` by LCEL

In [13]:
from operator import itemgetter
from langchain_community.memory.kg import ConversationKGMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:
{history}""",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationKGMemory(llm=llm, return_messages=True, memory_key="history")


class ConversationChain:
    def __init__(self, prompt, llm, memory):
        self.memory = memory
        self.chain = (
            RunnablePassthrough()
            | RunnablePassthrough.assign(
                history=RunnableLambda(memory.load_memory_variables)
                | itemgetter("history")
            )
            | prompt
            | llm
        )

    def invoke(self, input_dict):
        response = self.chain.invoke(input_dict)
        self.memory.save_context(input_dict, {"output": response.content})
        return response


conversation_with_kg = ConversationChain(prompt, llm, memory)

In [14]:
response = conversation_with_kg.invoke(
    {
        "input": "My name is Teddy. Shelly is a coworker of mine, and she's a new designer at our company."
    }
)
response.content

"Hello Teddy! It's nice to meet you. It sounds like you and Shelly work together at your company. As a new designer, Shelly must be excited to bring fresh ideas to the team. How are you finding working with her so far?"

In [15]:
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': [SystemMessage(content='On Shelly: Shelly is a coworker of Teddy. Shelly is a new designer at company.', additional_kwargs={}, response_metadata={})]}

In [16]:
conversation_with_kg.memory.clear()
conversation_with_kg.memory.load_memory_variables({"input": "who is Shelly?"})

{'history': []}